In [1]:
import pandas as pd
import datetime as dt
import re
import time
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
bets = pd.read_csv('Bets/Bets_2022-09-24.csv', index_col = 0)
bets.drop(0, axis = 0, inplace = True)
bets.reset_index(inplace = True, drop = True)
bets.to_csv('Bets/Bets_2022-09-24.csv')